In [127]:
import numpy as np 
import pandas as pd 

train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Columns of the dataset:
* *Survived*
* **Sex**
* **Pclass**
* Name
* **Age**
* **SibSp**
* **Parch**
* Ticket
* **Fare**
* Cabin
* Embarked

Taking bold ones as features on intuition that they are predictive


In [128]:
# find missing values
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [129]:
test_data.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [130]:
# replace missing ages with mean age/fare
train_data['Age'] = train_data['Age'].fillna(value=train_data['Age'].mean())
test_data['Age'] = test_data['Age'].fillna(value=test_data['Age'].mean())
test_data['Fare'] = test_data['Fare'].fillna(value=test_data['Fare'].mean())

In [131]:
from sklearn.model_selection import train_test_split

features = ['Sex', 'Pclass', 'Age', 'SibSp', 'Fare', 'Parch']
y = train_data['Survived']

# one-hot encode gender and align test/train encodings
onehot_X_train = pd.get_dummies(train_data[features])
onehot_X_test = pd.get_dummies(test_data[features])
X, X_test = onehot_X_train.align(onehot_X_test, join='left', axis=1)

train_X, val_X, train_y, val_y = train_test_split(X, y)

In [144]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf_model = RandomForestClassifier()
rf_model.fit(train_X, train_y)

rf_val_pred = rf_model.predict(val_X)
rf_acc = accuracy_score(rf_val_pred, val_y)
print(rf_acc)

0.8251121076233184


In [146]:
from sklearn.ensemble import GradientBoostingClassifier

gb_model = GradientBoostingClassifier()
gb_model.fit(train_X, train_y)

gb_val_pred = gb_model.predict(val_X)
gb_acc = accuracy_score(gb_val_pred, val_y)
print(gb_acc)

0.8340807174887892


In [147]:
gb_model.fit(X, y)

GradientBoostingClassifier()

In [148]:
test_data['Prediction'] = gb_model.predict(X_test)
test_data['Survived'] = test_data['Prediction']
submission = test_data[['PassengerId', 'Survived']]
submission.to_csv('submission.csv', index=False)